In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
# from sklearn.impute import SimpleImputer
# import os.path
# Harmonization
# from sklearn.model_selection import train_test_split
# from neurocombat_sklearn import CombatModel
# from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit, fit_and_eval

In [2]:
# scale = 1/5 
# scale = 1/10
# scale = 1/20 
# scale = 1/40

In [3]:
# def tf_aug(X_in, scaler=scale):
#     X_out = np.zeros_like(X_in)
#     for channel in range(X_in.shape[-1]):
#         this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
#         scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
#         this_X = jitter(this_X, sigma=scale)
#         this_X = scaling(this_X, sigma=scale)
#         this_X = time_warp(this_X, sigma=scale)
#         this_X = window_warp(this_X, window_ratio=scale)
#         X_out[..., channel] = this_X[0, ..., 0]
#     return X_out


# def augment_this(X_in, y_in):
#     X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
#     return X_out, y_in


In [4]:
X, y, site = load_data()

2022-07-26 00:52:17.150917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:52:17.264964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:52:17.265355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:52:17.269202: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1503, 1334, 1576, 469, 1795])

In [8]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_1_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 72)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 6)            1302      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 6)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 16)            304       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 26)            1274      
________________________________

2022-07-26 00:52:26.656224: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-26 00:52:28.616089: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-07-26 00:52:30.812661: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 00:52:30.813178: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 00:52:30.813219: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-07-26 00:52:30.813913: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 00:52:30.814011: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx co

37/37 [==============================] - 9s 16ms/step - loss: 38.2972 - mean_squared_error: 38.2972 - rmse: 6.1885 - mean_absolute_error: 4.9265 - val_loss: 15.6233 - val_mean_squared_error: 15.6233 - val_rmse: 3.9526 - val_mean_absolute_error: 2.9445
Epoch 2/1000
37/37 [==============================] - 0s 6ms/step - loss: 19.7370 - mean_squared_error: 19.7370 - rmse: 4.4426 - mean_absolute_error: 3.4959 - val_loss: 11.0136 - val_mean_squared_error: 11.0136 - val_rmse: 3.3187 - val_mean_absolute_error: 2.4761
Epoch 3/1000
37/37 [==============================] - 0s 6ms/step - loss: 19.1818 - mean_squared_error: 19.1818 - rmse: 4.3797 - mean_absolute_error: 3.4510 - val_loss: 11.7199 - val_mean_squared_error: 11.7199 - val_rmse: 3.4234 - val_mean_absolute_error: 2.5233
Epoch 4/1000
37/37 [==============================] - 0s 6ms/step - loss: 17.2959 - mean_squared_error: 17.2959 - rmse: 4.1588 - mean_absolute_error: 3.2698 - val_loss: 9.6904 - val_mean_squared_error: 9.6904 - val_rmse:

In [9]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_2_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 100, 6)            1302      
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 50, 6)             0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 50, 16)            304       
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 25, 16)            0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 25, 26)            1274      
_____________________________

In [ ]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model,
            model_dict,
            X, 
            y,
            site,
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_3_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_80 (Conv1D)           (None, 100, 6)            1302      
_________________________________________________________________
max_pooling1d_80 (MaxPooling (None, 50, 6)             0         
_________________________________________________________________
conv1d_81 (Conv1D)           (None, 50, 16)            304       
_________________________________________________________________
max_pooling1d_81 (MaxPooling (None, 25, 16)            0         
_________________________________________________________________
conv1d_82 (Conv1D)           (None, 25, 26)            1274      
_____________________________

In [ ]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model,
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_4_pred.csv")

In [ ]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_5_pred.csv")

In [ ]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_6_pred.csv")

In [ ]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)